In [22]:
import torch
import torch.nn as nn
import torchvision

from torchvision.models import resnet50
from thop import profile
from thop import clever_format

from torchsummary import summary

## Basic tutorial

In [25]:
model = resnet50()
input = torch.randn(1, 3, 224, 224)
flops, params = profile(model, inputs=(input, ), verbose=False)

In [26]:
summary(model.cuda(), input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [27]:
flops, params

(4142111744.0, 25557032.0)

In [28]:
flops, params = clever_format([flops, params], "%.3f")

In [29]:
flops, params

('4.142G', '25.557M')

In [30]:
# nn.DataParallel 에 태워놓고 FLOPs 계산하려하면 에러남.
model = resnet50().cuda()
model = nn.DataParallel(model)
input = torch.randn(1, 3, 224, 224).cuda()
flops, params = profile(model, inputs=(input, ), verbose=False)

RuntimeError: module must have its parameters and buffers on device cuda:0 (device_ids[0]) but found one of them on device: cpu

## RWNN 모델 적용

In [19]:
import os
import time
import logging

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn    # for hardware tunning (cudnn.benchmark = True)

from tensorboardX import SummaryWriter

from model import CNN
from lr_scheduler import LRScheduler

In [47]:
###########################
# 1. Configuration
########################### 
class Args():
    def __init__(self, run_code='default_run_code'):
        self.run_code = run_code
        self.workers = 2
        self.batch_size = 128
        self.epochs = 100

        self.lr_mode = 'cosine'
        self.warmup_mode = 'linear'    # default
        self.base_lr = 0.1
        
        self.warmup_epochs = 0
        self.warmup_lr = 0.0
        self.targetlr = 0.0

        self.momentum = 0.9
        self.weight_decay = 0.00005

        self.net_type = 'regular'
        self.channels = 109
        self.nodes = 32
        self.graph_model = 'WS'
        self.K = 4
        self.P = 0.75
        
        self.resume = False    # checkpoint에 저장된 모델 불러와서 사용할지 여부

        self.seed = 1

        self.print_freq = 100
        
        self.model_dir = './checkpoint' + '/' + run_code + '/'    # graph 정보 (e.g. conv2.yaml 등), 
        self.tensorboard_path = './tensorboard/' + run_code + '/'
        self.log_path = './logs/' + run_code + '/'

        if not os.path.isdir(self.model_dir): os.makedirs(self.model_dir)
        if not os.path.isdir(self.tensorboard_path): os.makedirs(self.tensorboard_path)
        if not os.path.isdir(self.log_path): os.makedirs(self.log_path)        

        logging.basicConfig(filename=self.log_path + 'logging.log', level=logging.INFO)
        logging.info('Configuration initialized.')


In [48]:
args = Args(run_code='regular_c109_n32_2')    # default source
print(args.model_dir)

./checkpoint/regular_c109_n32_2/


In [49]:
model = CNN(args)

In [56]:
print(model)

CNN(
  (conv1): depthwise_separable_conv_3x3(
    (depthwise): Conv2d(3, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=3)
    (pointwise): Conv2d(3, 54, kernel_size=(1, 1), stride=(1, 1))
  )
  (bn1): BatchNorm2d(54, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): StageBlock(
    (nodeop): ModuleList(
      (0): Node_OP(
        (conv): Triplet_unit(
          (relu): ReLU()
          (conv): depthwise_separable_conv_3x3(
            (depthwise): Conv2d(54, 54, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=54)
            (pointwise): Conv2d(54, 109, kernel_size=(1, 1), stride=(1, 1))
          )
          (bn): BatchNorm2d(109, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): Node_OP(
        (conv): Triplet_unit(
          (relu): ReLU()
          (conv): depthwise_separable_conv_3x3(
            (depthwise): Conv2d(54, 54, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=54)
  

In [50]:
input = torch.randn(1, 3, 224, 224)
flops, params = profile(model, inputs=(input, ))

Register FLOP counter for module Conv2d(3, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=3)
Register FLOP counter for module Conv2d(3, 54, kernel_size=(1, 1), stride=(1, 1))
Register FLOP counter for module BatchNorm2d(54, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Register FLOP counter for module ReLU()
Register FLOP counter for module Conv2d(54, 54, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=54)
Register FLOP counter for module Conv2d(54, 109, kernel_size=(1, 1), stride=(1, 1))
Register FLOP counter for module BatchNorm2d(109, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Register FLOP counter for module ReLU()
Register FLOP counter for module Conv2d(54, 54, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=54)
Register FLOP counter for module Conv2d(54, 109, kernel_size=(1, 1), stride=(1, 1))
Register FLOP counter for module BatchNorm2d(109, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Re

In [51]:
print(flops, params)

flops, params = clever_format([flops, params], "%.3f")
print(flops, params)

7783212032.0 31619008.0
7.783G 31.619M


In [54]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## args.net_type = 'small' 과 비교

In [57]:
###########################
# 1. Configuration
########################### 
class Args():
    def __init__(self, run_code='default_run_code'):
        self.run_code = run_code
        self.workers = 2
        self.batch_size = 128
        self.epochs = 100

        self.lr_mode = 'cosine'
        self.warmup_mode = 'linear'    # default
        self.base_lr = 0.1
        
        self.warmup_epochs = 0
        self.warmup_lr = 0.0
        self.targetlr = 0.0

        self.momentum = 0.9
        self.weight_decay = 0.00005

        self.net_type = 'small'
        self.channels = 109
        self.nodes = 32
        self.graph_model = 'WS'
        self.K = 4
        self.P = 0.75
        
        self.resume = False    # checkpoint에 저장된 모델 불러와서 사용할지 여부

        self.seed = 1

        self.print_freq = 100
        
        self.model_dir = './checkpoint' + '/' + run_code + '/'    # graph 정보 (e.g. conv2.yaml 등), 
        self.tensorboard_path = './tensorboard/' + run_code + '/'
        self.log_path = './logs/' + run_code + '/'

        if not os.path.isdir(self.model_dir): os.makedirs(self.model_dir)
        if not os.path.isdir(self.tensorboard_path): os.makedirs(self.tensorboard_path)
        if not os.path.isdir(self.log_path): os.makedirs(self.log_path)        

        logging.basicConfig(filename=self.log_path + 'logging.log', level=logging.INFO)
        logging.info('Configuration initialized.')


In [58]:
args = Args(run_code='regular_c109_n32')    # default source
print(args.model_dir)

./checkpoint/regular_c109_n32/


In [59]:
model = CNN(args)

In [60]:
print(model)

CNN(
  (conv1): depthwise_separable_conv_3x3(
    (depthwise): Conv2d(3, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=3)
    (pointwise): Conv2d(3, 54, kernel_size=(1, 1), stride=(1, 1))
  )
  (bn1): BatchNorm2d(54, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Triplet_unit(
    (relu): ReLU()
    (conv): depthwise_separable_conv_3x3(
      (depthwise): Conv2d(54, 54, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=54)
      (pointwise): Conv2d(54, 109, kernel_size=(1, 1), stride=(1, 1))
    )
    (bn): BatchNorm2d(109, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): StageBlock(
    (nodeop): ModuleList(
      (0): Node_OP(
        (conv): Triplet_unit(
          (relu): ReLU()
          (conv): depthwise_separable_conv_3x3(
            (depthwise): Conv2d(109, 109, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=109)
            (pointwise): Conv2d(109, 109, kernel_size=(1, 1), stride=(1, 1

In [61]:
input = torch.randn(1, 3, 224, 224)
flops, params = profile(model, inputs=(input, ))

Register FLOP counter for module Conv2d(3, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=3)
Register FLOP counter for module Conv2d(3, 54, kernel_size=(1, 1), stride=(1, 1))
Register FLOP counter for module BatchNorm2d(54, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Register FLOP counter for module ReLU()
Register FLOP counter for module Conv2d(54, 54, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=54)
Register FLOP counter for module Conv2d(54, 109, kernel_size=(1, 1), stride=(1, 1))
Register FLOP counter for module BatchNorm2d(109, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Register FLOP counter for module ReLU()
Register FLOP counter for module Conv2d(109, 109, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=109)
Register FLOP counter for module Conv2d(109, 109, kernel_size=(1, 1), stride=(1, 1))
Register FLOP counter for module BatchNorm2d(109, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True

Register FLOP counter for module ReLU()
Register FLOP counter for module Conv2d(436, 436, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=436)
Register FLOP counter for module Conv2d(436, 436, kernel_size=(1, 1), stride=(1, 1))
Register FLOP counter for module BatchNorm2d(436, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
THOP has not implemented counting method for  Sigmoid()
Register FLOP counter for module ReLU()
Register FLOP counter for module Conv2d(436, 436, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=436)
Register FLOP counter for module Conv2d(436, 436, kernel_size=(1, 1), stride=(1, 1))
Register FLOP counter for module BatchNorm2d(436, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
THOP has not implemented counting method for  Sigmoid()
Register FLOP counter for module ReLU()
Register FLOP counter for module Conv2d(436, 436, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=436)
Register FLOP counter for modul

In [62]:
print(flops, params)

flops, params = clever_format([flops, params], "%.3f")
print(flops, params)

1835179264.0 9296907.0
1.835G 9.297M
